In [21]:
%pip install pfapack

Note: you may need to restart the kernel to use updated packages.


In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
from pfapack import pfaffian
import math

In [23]:
def create_u_std_matrix(N1, N2, bc1, bc2):
    N = N1 * N2 # N1 为 a_1 方向 unit cell 数量
    M = np.zeros((2 * N, 2 * N)) # M 储存 u_jk(j 属于 A 子格，k 属于 B 子格)构型
    u = np.zeros((2 * N, 2 * N))
    
    if bc1 == "PBC":
        sign1 = 1
    elif bc1 == "APBC":
        sign1 = -1
    if bc2 == "PBC":
        sign2 = 1
    elif bc2 == "APBC":
        sign2 = -1    
    
    def index(n1, n2, sub):
        return n1 + n2 * N1 + N1 * N2 * sub # A子格对应 sub=0,B子格对应 sub=1
    
    for n2 in range(N2):
        for n1 in range(N1):
            j = index(n1, n2, 0) # j 为(n1, n2)unit cell 内 A 子格格点的单指标
            
            kx = index(n1, n2, 1) # kx 为(n1, n2)unit cell 内 B 子格格点的单指标
            M[j, kx] = 1 # j 与 kx 由 x-bond 相连，一定不跨越边界
            
            ky = index((n1 + 1) % N1, n2, 1) # ky 为 (n1, n2) 往 a1 方向走一步得到的 unit cell 内 B 子格格点的单指标j 与 ky 由 y-bond 相连
            if n1 == N1 - 1:
                M[j, ky] = sign1
            else:
                M[j, ky] = 1

            kz = index(n1, (n2 + 1) % N2, 1) # kz 为 (n1, n2) 往 a2 方向走一步得到的 unit cell 内 B 子格格点的单指标，j 与 kz 由 z-bond 相连
            if n2 == N2 - 1:
                M[j, kz] = sign2
            else:
                M[j, kz] = 1
      
    u = M - M.T                        
    return u

In [24]:
def create_t_matrix(N1, N2, Kx, Ky, Kz, u):
    N = N1 * N2
    t = np.zeros((N, N))
    K = np.array([Kx, Ky, Kz])
    for i in range(N):
        for j in range(N):
            sum = 0
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            
            for delta_vec in np.array([[0, 0], [1, 0], [0, 1]]):
                tmp1 = (r_j_vec[0] - delta_vec[0] + N1) % N1
                tmp2 = (r_j_vec[1] - delta_vec[1] + N2) % N2
                if (np.array_equal(r_i_vec, np.array([tmp1, tmp2]))):
                    delta_func = 1
                else:
                    delta_func = 0
                    
                tmp3 = delta_vec[0] + delta_vec[1] * 2
                K_delta = K[tmp3]

                sum += delta_func * K_delta
                
            sum *= u[i, j + N]
            t[i, j] = sum
    return t

In [25]:
def create_t_plus_matrix(N1, N2, u):
    N = N1 * N2
    t_plus = np.zeros((N, N))
    def index1(UC_vec, sub):
        return N1 * UC_vec[1] + UC_vec[0] + sub * N
    list1 = [np.array([1, 0]), np.array([0, 1]), np.array([0, 0])]
    list2 = [np.array([0, 1]), np.array([0, 0]), np.array([1, 0])]
    for i in range(N):
        for j in range(N):
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            sum = 0
            
            for delta1_vec, delta2_vec in zip(list1, list2):
                tmp1 = (r_i_vec[0]-delta1_vec[0] + N1) % N1 # r_i - delta_1 的 a_1 分量，考虑了周期性边界条件
                tmp2 = (r_i_vec[1]-delta1_vec[1] + N2) % N2 # r_i - delta_1 的 a_2 分量，考虑了周期性边界条件
                tmp3 = (r_j_vec[0]-delta2_vec[0] + N1) % N1 # r_j - delta_2 的 a_1 分量，考虑了周期性边界条件
                tmp4 = (r_j_vec[1]-delta2_vec[1] + N2) % N2 # r_j - delta_2 的 a_2 分量，考虑了周期性边界条件
                if (np.array_equal(np.array([tmp1, tmp2]) , np.array([tmp3, tmp4]))):
                    delta_func = 1
                else:
                    delta_func = 0
                sum += delta_func * u[index1(r_i_vec, 1), index1(np.array([tmp1, tmp2]), 0) ] * u[index1(r_j_vec, 1), index1(np.array([tmp3, tmp4]), 0)]

            t_plus[i][j] = sum
            
    return t_plus

In [26]:
def create_t_minus_matrix(N1, N2, u):
    N = N1 * N2
    t_minus = np.zeros((N, N))
    def index1(UC_vec, sub):
        return N1 * UC_vec[1] + UC_vec[0] + sub * N
    list1 = [np.array([-1, 0]), np.array([0, -1]), np.array([0, 0])]
    list2 = [np.array([0, -1]), np.array([0, 0]), np.array([-1, 0])]
    for i in range(N):
        for j in range(N):
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            sum = 0
            
            for delta1_vec, delta2_vec in zip(list1, list2):
                tmp1 = (r_i_vec[0]-delta1_vec[0] ) % N1 # r_i - delta_1 的 a_1 分量，考虑了周期性边界条件
                tmp2 = (r_i_vec[1]-delta1_vec[1] ) % N2 # r_i - delta_1 的 a_2 分量，考虑了周期性边界条件
                tmp3 = (r_j_vec[0]-delta2_vec[0] ) % N1 # r_j - delta_2 的 a_1 分量，考虑了周期性边界条件
                tmp4 = (r_j_vec[1]-delta2_vec[1] ) % N2 # r_j - delta_2 的 a_2 分量，考虑了周期性边界条件
                if (np.array_equal(np.array([tmp1, tmp2]) , np.array([tmp3, tmp4]))):
                    delta_func = 1
                else:
                    delta_func = 0
                    
                sum += delta_func * u[index1(r_i_vec, 0), index1(np.array([tmp1, tmp2]), 1) ] * u[index1(r_j_vec, 0), index1(np.array([tmp3, tmp4]), 1)]

            t_minus[i][j] = sum
            
    return t_minus

In [27]:
def create_P_times_P_prime_matrix(N):
    P = np.zeros((2 * N, 2* N))
    for i in range(N):
        P[i, 2 * i] = 1
        P[i + N, 2 * i + 1] = 1
    # 计算 P_prime 矩阵
    I_N = np.eye(N) # N \times N 单位矩阵
    P_prime = np.kron(I_N, 0.5 * np.array([[1, 1j], [1, -1j]]))
    #print("P_prime:\n", P_prime)
    P_times_P_prime = np.dot(P, P_prime) # 矩阵乘法
    return P_times_P_prime

In [1]:
def add_vison_pair(N1, N2, n1, n2, bond, u):
    def index(n1, n2, sub):
        return n1 + n2 * N1 + N1 * N2 * sub # A子格对应 sub=0,B子格对应 sub=1
    j = index(n1, n2, 0)
    if bond == 1: # x-bond
        k = index(n1, n2, 1)
    elif bond == 2: # y-bond
        k = index((n1+1)%N1, n2, 1)
    elif bond == 3: # z-bond
        k = index(n1, (n2+1)%N2, 1)

    u[j, k] = -u[j, k]
    u[k, j] = -u[k, j]
    return u
    

In [ ]:
def calc_hopping_amplitude(N1, N2, n1, n2, bond_1, bond_2, h_mag, ):
    
    

In [29]:
N1, N2, Kx, Ky, Kz, kappa = 10, 10, 1, 1, 1, 0.1
h = np.array([0.1, 0.1, 0.1]) # 磁场
N = N1 * N2
n1, n2 = N1//2, N2//2 
vison_bond_1 = 1 # 1对应x-bond, 2对应y-bond, 3对应z-bond
vison_bond_2 = 2

In [30]:
results_H0 = {} # 空字典，以 (bc1, bc2) 为键，每个键储存对应边界条件下 H0 的本征值、本征矢量、基态能量
results_H_total = {} # 空字典，以 (bc1, bc2) 为键，每个键储存对应边界条件下 H_total = H0 + H_kappa 的本征值、本征矢量、基态能量
P_times_P_prime = create_P_times_P_prime_matrix(N)
for bc1 in ["PBC", "APBC"]:
    for bc2 in ["PBC", "APBC"]:
        u_std = create_u_std_matrix(N1, N2, bc1, bc2)
        #print('u_std:\n', u_std)
        u_1 = add_vison_pair(N1, N2, n1, n2, vison_bond_1, u_std)
        u_2 = add_vison_pair(N1, N2, n1, n2, vison_bond_2, u_std)
        #print('u_add_vison_pair:\n', u)
        
        t_std = create_t_matrix(N1, N2, Kx, Ky, Kz, u_std)
        Xi_std = t_std + t_1.std
        Delta_std = -(t_std - t_std.T)
        H_std = np.block([
            [Xi_std, Delta_std],
            [-Delta_std, -Xi_std.T]
        ])
        eigvals_std, eigvecs_std = np.linalg.eigh(H_std)
        
        t_1 = create_t_matrix(N1, N2, Kx, Ky, Kz, u_1)
        Xi_1 = t_1 + t_1.T
        Delta_1 = -(t_1 - t_1.T)
        H_1 = np.block([
            [Xi_1, Delta_1],
            [-Delta_1, -Xi_1.T]
        ])
        eigvals_1, eigvecs_1 = np.linalg.eigh(H_1)

        t_2 = create_t_matrix(N1, N2, Kx, Ky, Kz, u_2)
        Xi_2 = t_2 + t_2.T
        Delta_2 = -(t_2 - t_2.T)
        H_2 = np.block([
            [Xi_2, Delta_2],
            [-Delta_2, -Xi_2.T]
        ])
        eigvals_2, eigvecs_2 = np.linalg.eigh(H_2)

        hopping_amplitude = np.zeros(3) # 0对应y-z跃迁，1对应z-x跃迁，2对应x-y跃迁
        hoppong_amplitude = 

        
        #ground_state_energy = np.sum(eigvals[:N]) / 2 # 计算无磁场时基态能量
        
        # A_prime = -1j * np.linalg.multi_dot([P_times_P_prime.T.conj(), H0, P_times_P_prime])
        # A = (A_prime - A_prime.T) / 2
        # A = np.real(A) # A 是实矩阵
        # #print("A_matrix:\n", A)
        # pf = pfaffian.pfaffian(A)
        # parity = np.sign(pf)
        # results_H0[(bc1, bc2)] = {
        #     "eigvals": eigvals,
        #     "eigvecs": eigvecs,
        #     "ground_state_energy": ground_state_energy,
        #     "pf": pf,
        #     "parity": parity
        # }
#         ############ 有磁场 ############
#         # 计算磁场项哈密顿量
#         t_plus = create_t_plus_matrix(N1, N2, u)
#         t_minus = create_t_minus_matrix(N1, N2, u)
#         Xi_prime = np.zeros((N, N), dtype = complex)
#         Delta_prime = np.zeros((N, N), dtype = complex)         
#         Xi_prime = 1j * kappa * (t_plus + t_minus - (t_plus + t_minus).T)
#         Delta_prime = 1j * kappa * ((t_minus - t_plus) - (t_minus - t_plus).T)
#         #print("Delta_prime:", Delta_prime)
#         #C_prime = -1j * kappa * np.trace(t_plus + t_minus)

#         H_kappa = np.block([
#             [Xi_prime, Delta_prime],
#             [Delta_prime, Xi_prime]
#         ])
#         H_total = H0 + H_kappa # 总哈密顿量
#         #print("H_total", H_total)
#         eigvals, eigvecs = np.linalg.eigh(H_total)
#         ground_state_energy = np.sum(eigvals[:N]) / 2 # 计算有磁场时体系基态能量

#         # 计算 A_prime 矩阵
#         A_prime = -1j * np.linalg.multi_dot([P_times_P_prime.T.conj(), H_total, P_times_P_prime])
#         A = (A_prime - A_prime.T) / 2
#         #A = np.real(A) # A 是实矩阵
#         #print("A_matrix:\n", A)
#         pf = pfaffian.pfaffian(A)
#         #print("Pfaffian_of_A_matrix:", pf)
#         #print("det of A:", np.linalg.det(A))
#         #print("larger pf:", pf * 2**N * math.factorial(N))
#         parity = np.sign(pf)
#         results_H_total[(bc1, bc2)] = {
#             "eigvals": eigvals,
#             "eigvecs": eigvecs,
#             "ground_state_energy": ground_state_energy,
#             "pf": pf,
#             "parity": parity
#         }
#         #print(eigvals)

# print(f"config:N1={N1},N2={N2},Kx={Kx},Ky={Ky},Kz={Kz},kappa={kappa},vison_at({n1},{n2}),vison_bond={vison_bond}\n")
# for key in results_H0:
#     print(f"(bc1, bc2)={key}:")
#     print(f"H0:ground_state_energy = {results_H0[key]['ground_state_energy']}", )
#     print(f"H0:pf = {results_H0[key]['pf']:}")
#     print(f"H0:parity = {results_H0[key]['parity']}\n")
# for key in results_H_total:
#     print(f"(bc1, bc2)={key}:")
#     print(f"H_total:ground_state_energy = {results_H_total[key]['ground_state_energy']}")
#     print(f"H_total:pf = {results_H_total[key]['pf']:}", )
#     print(f"H_total:parity = {results_H_total[key]['parity']}\n", )

config:N1=10,N2=10,Kx=1,Ky=1,Kz=1,kappa=0.1,vison_at(5,5),vison_bond=1

(bc1, bc2)=('PBC', 'PBC'):
H0:ground_state_energy = -157.26147532296804
H0:pf = -70245619662580.88
H0:parity = -1.0

(bc1, bc2)=('PBC', 'APBC'):
H0:ground_state_energy = -157.1612992151127
H0:pf = 25072235821874.867
H0:parity = 1.0

(bc1, bc2)=('APBC', 'PBC'):
H0:ground_state_energy = -157.16129921511273
H0:pf = 25072235821870.695
H0:parity = 1.0

(bc1, bc2)=('APBC', 'APBC'):
H0:ground_state_energy = -157.21007743148215
H0:pf = 50033278746885.54
H0:parity = 1.0

(bc1, bc2)=('PBC', 'PBC'):
H_total:ground_state_energy = -160.89489862079523
H_total:pf = (-1.3783737079409116e+16+0j)
H_total:parity = (-1+0j)

(bc1, bc2)=('PBC', 'APBC'):
H_total:ground_state_energy = -160.89280816774965
H_total:pf = (1.358759956146004e+16-0j)
H_total:parity = (1+0j)

(bc1, bc2)=('APBC', 'PBC'):
H_total:ground_state_energy = -160.89280816774962
H_total:pf = (1.3587599561460026e+16+0j)
H_total:parity = (1+0j)

(bc1, bc2)=('APBC', 'APBC'):
